In [1]:
#2 # 2 in spark
# Create a new Spark notebook called labd.ipynb. Write (or copy and edit) Spark code to 
# set up the Spark session. Make sure your Spark session supports Minio access and 
# include the hadoop-aws Spark Jar package. Provide a screenshot of your code and the output.

import pyspark
from pyspark.sql import SparkSession
print("hello")

hello


In [2]:
# MINIO CONFIGURATION
s3_host = "minio"
s3_url = f"http://{s3_host}:9000"
s3_key = "minio"
s3_secret = "SU2orange!"
s3_bucket = "labe"
print(s3_url)

http://minio:9000


In [3]:
# Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2")\
    .config("spark.hadoop.fs.s3a.endpoint", s3_url) \
    .config("spark.hadoop.fs.s3a.access.key", s3_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret) \
    .config("spark.hadoop.fs.s3a.fast.upload", True) \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext
print(s3_bucket)
#sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d5be1d0-082c-4843-8950-a53b66224ea6;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.1.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.1.2/hadoop-aws-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.1.2!hadoop-aws.jar (99ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.271/aws-java-sdk-bundle-1.11.271.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.11.271!aws-java-sdk-bundle.jar (12395ms)
:: resolution report :: resolve 3227ms :: artifacts dl 12501ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.271 from central in [default]
	org.apache.hadoop#hadoop-aws;3.1.2 from central in [de

labe


In [5]:
# 5
#Configure Spark to read from Minio labe bucket, then load 
#syracuse-ny.csv into a DataFrame and register  it as the table weather. 
s3_bucket ="labe"
file_name = "syracuse-ny.csv"
print("hello")
source = f"s3a://{s3_bucket}/{file_name}"
weather = spark.read.option("header", True).option("inferSchema", True)\
    .csv(source)
weather.show()


hello
+---------+----------------+-----------------+----------------+--------------+--------------+-------------+------------+-------------+------------+------------------------+-------------------------+------------------------+-------------------+--------------------+-------------------+-----------------+------------------+-----------------+---------------+----------+-------------+--------------+
|      EST|Max TemperatureF|Mean TemperatureF|Min TemperatureF|Max Dew PointF|MeanDew PointF|Min DewpointF|Max Humidity|Mean Humidity|Min Humidity|Max Sea Level PressureIn|Mean Sea Level PressureIn|Min Sea Level PressureIn|Max VisibilityMiles|Mean VisibilityMiles|Min VisibilityMiles|Max Wind SpeedMPH|Mean Wind SpeedMPH|Max Gust SpeedMPH|PrecipitationIn|CloudCover|       Events|WindDirDegrees|
+---------+----------------+-----------------+----------------+--------------+--------------+-------------+------------+-------------+------------+------------------------+-------------------------+----

In [7]:
weather = spark.read.option("header", True).option("inferSchema", True)\
    .csv(source)
weather.createOrReplaceTempView("weather")

spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |  weather|       true|
+--------+---------+-----------+



In [11]:
# 6 Rewrite Question 2 using pure Spark SQL and the weather temp view. NOTE: There will be some subtle 
#differences with how you must write the code, so be sure to printSchema() so you can see what the 
# columns are. 
query = '''
WITH
Source 
AS
(
	SELECT 	cast(split(EST, '-')[0] AS int) AS year, 
	        cast(split(EST, '-')[1] AS int) AS month, 
		    cast(`Min TemperatureF` AS int) as mintemp, 
		    cast(`Max TemperatureF` AS int) AS maxtemp
	FROM weather
)
SELECT year, month, AVG(mintemp) AS avgmintemp, AVG(maxtemp) AS avgmaxtemp
FROM Source
GROUP BY year, month
ORDER BY year, month
'''
spark.sql(query).printSchema()
weather2 = spark.sql(query)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- avgmintemp: double (nullable = true)
 |-- avgmaxtemp: double (nullable = true)



In [13]:
# 7 Save the output from the DataFrame in Question 6 to the temp view 
# monthly_syracuse_weather_averages. Prove the view is there by querying it.

weather2.createOrReplaceTempView("monthly_syracuse_weather_averages")
query2 = '''
SELECT * FROM monthly_syracuse_weather_averages
'''
spark.sql(query2).show()

+----+-----+------------------+------------------+
|year|month|        avgmintemp|        avgmaxtemp|
+----+-----+------------------+------------------+
|1997|    1|15.774193548387096| 31.64516129032258|
|1997|    2|22.607142857142858|37.785714285714285|
|1997|    3|25.032258064516128| 41.12903225806452|
|1997|    4| 34.43333333333333|              54.1|
|1997|    5|43.096774193548384| 61.58064516129032|
|1997|    6|              57.8|              78.4|
|1997|    7| 59.87096774193548| 80.19354838709677|
|1997|    8| 58.70967741935484| 78.38709677419355|
|1997|    9| 51.06666666666667|              69.4|
|1997|   10|38.935483870967744| 59.16129032258065|
|1997|   11|31.466666666666665|42.666666666666664|
|1997|   12|25.032258064516128|35.516129032258064|
|1998|    1|23.258064516129032| 35.41935483870968|
|1998|    2|24.142857142857142|              38.0|
|1998|    3|30.129032258064516|45.064516129032256|
|1998|    4|              38.0| 58.46666666666667|
|1998|    5| 52.45161290322581|

In [18]:
# 8. CHALLENGE YOURSELF! At the bottom of the work/content/E-Drill-Spark.ipynb file there is a section 
#called Big Data to Small Data. Try to write a complete program that:
# a. Inputs a month 1–12 at run-time
# b. Displays a scatter plot of min/max average monthly temperatures, where year is on the X-axis
from IPython.display import display, HTML
from ipywidgets import interact_manual

display(HTML("<H1>Syracuse Weather</h1>"))
@interact_manual(Month=(1,12))
def showResult(Month):
    df = spark.sql(f"select * from monthly_syracuse_weather_averages WHERE month = {Month}").toPandas()
    display(df)
    display(df.plot(x="year", y="avgmintemp"))

interactive(children=(IntSlider(value=6, description='Month', max=12, min=1), Button(description='Run Interact…

In [20]:
import matplotlib.pyplot as plt
display(HTML("<H1>Syracuse Weather</h1>"))
@interact_manual(Month=(1,12))
def showResult2(Month):
    df = spark.sql(f"select * from monthly_syracuse_weather_averages WHERE month = {Month}").toPandas()
    df.set_index("year")
    plt.figure(figsize=(15,10))
    plt.scatter(df["year"],y=df["avgmintemp"], label='monthly avg min', marker='v')
    plt.scatter(df["year"],y=df["avgmaxtemp"], label='monthly avg max', marker='^')
    plt.legend(title=f"Temperatures for Month {Month}")
    plt.show()

interactive(children=(IntSlider(value=6, description='Month', max=12, min=1), Button(description='Run Interact…